In [9]:
import cv2
import numpy as np
import os
import matplotlib
%matplotlib inline

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks.callbacks import ModelCheckpoint
from keras.layers.pooling import AveragePooling2D, MaxPooling2D
import tensorflow as tf
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input, Convolution2D, BatchNormalization
from keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report 
from imutils import paths
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
import argparse
import pickle
import cv2
import os   
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.metrics import accuracy_score, recall_score, precision_score
from keras.models import load_model

In [10]:
from keras.layers import LSTM

In [ ]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 15, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

### Prepare data

In [3]:
path_train = '../Datasets/Videos/lstm/train/'
path_test = '../Datasets/Videos/lstm/test/'

In [47]:
X_train = []
X_test = []

y_train = []
y_test = []

In [48]:
for file in os.listdir(path_train):
    if file.endswith(".csv"):
        data = pd.read_csv(path_train + file).drop('Unnamed: 0', axis=1)
        pos = 9
        while pos < data.shape[0]:
            X_train.append(data.drop('label', axis=1).values[pos-9: pos+1, :])
            y_train.append(data['label'].iloc[pos])
            pos += 1

In [49]:
for file in os.listdir(path_test):
    if file.endswith(".csv"):
        data = pd.read_csv(path_test + file).drop('Unnamed: 0', axis=1)
        pos = 9
        while pos < data.shape[0]:
            X_test.append(data.drop('label', axis=1).values[pos-9: pos+1, :])
            y_test.append(data['label'].iloc[pos])
            pos += 1

In [50]:
X_train = np.array(X_train)
X_test = np.array(X_test)

def label_to_float(x):
    return 0.0 if x == 'fire' else 1.0

y_train = np.array([label_to_float(x) for x in y_train])
y_test = np.array([label_to_float(x) for x in y_test])

In [8]:
scale = np.abs(X_train).max()

X_train /= scale
X_test /= scale

### Train model

In [51]:
n_timesteps = 10
n_features = 640

model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps, n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 100)               296400    
_________________________________________________________________
dropout_17 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 101       
Total params: 306,601
Trainable params: 306,601
Non-trainable params: 0
_________________________________________________________________


In [52]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=30, batch_size=64, verbose=1)

Epoch 1/30
1676/1676 [==============================] - 3s 2ms/step - loss: 0.6727 - accuracy: 0.6020
Epoch 2/30
1676/1676 [==============================] - 2s 1ms/step - loss: 0.6119 - accuracy: 0.6838
Epoch 3/30
1676/1676 [==============================] - 2s 1ms/step - loss: 0.6164 - accuracy: 0.6850
Epoch 4/30
1676/1676 [==============================] - 3s 2ms/step - loss: 0.6115 - accuracy: 0.6897
Epoch 5/30
1676/1676 [==============================] - 2s 1ms/step - loss: 0.6032 - accuracy: 0.7005
Epoch 6/30
1676/1676 [==============================] - 2s 1ms/step - loss: 0.6014 - accuracy: 0.7130
Epoch 7/30
1676/1676 [==============================] - 2s 1ms/step - loss: 0.6203 - accuracy: 0.6963
Epoch 8/30
1676/1676 [==============================] - 2s 1ms/step - loss: 0.6206 - accuracy: 0.6879
Epoch 9/30
1676/1676 [==============================] - 2s 1ms/step - loss: 0.6208 - accuracy: 0.6945
Epoch 10/30
1676/1676 [==============================] - 2s 1ms/step - loss: 0.612

In [29]:
model.save('Trained models/LSTM #1')

In [53]:
results = model.evaluate(X_test, y_test, batch_size=64)
print('test loss, test acc:', results)

276/276 [==============================] - 0s 1ms/step
test loss, test acc: [0.43614496441854944, 0.9166666865348816]


In [54]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred[y_pred <= 0.5] = 0
y_pred[y_pred > 0.5] = 1

print(classification_report(y_test, y_pred))

276/276 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

         0.0       0.94      0.97      0.96       260
         1.0       0.00      0.00      0.00        16

    accuracy                           0.92       276
   macro avg       0.47      0.49      0.48       276
weighted avg       0.89      0.92      0.90       276



### 2 layers

In [39]:
n_timesteps = 10
n_features = 640

model2 = Sequential()
model2.add(LSTM(100, input_shape=(n_timesteps, n_features), return_sequences=True))
model2.add(Dropout(0.5))
model2.add(LSTM(200, return_sequences=False))
model2.add(Dropout(0.5))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 10, 100)           296400    
_________________________________________________________________
dropout_15 (Dropout)         (None, 10, 100)           0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 200)               240800    
_________________________________________________________________
dropout_16 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 101       
Total params: 557,401
Trainable params: 557,401
Non-trainable params: 0
________________________________________________

In [40]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

hist2 = model2.fit(X_train, y_train, epochs=30, batch_size=64, verbose=1)

Epoch 1/30
1676/1676 [==============================] - 6s 3ms/step - loss: 0.5808 - accuracy: 0.7088
Epoch 2/30
1676/1676 [==============================] - 5s 3ms/step - loss: 0.5746 - accuracy: 0.7267
Epoch 3/30
1676/1676 [==============================] - 4s 2ms/step - loss: 0.5451 - accuracy: 0.7595
Epoch 4/30
1676/1676 [==============================] - 5s 3ms/step - loss: 0.5560 - accuracy: 0.7530
Epoch 5/30
1676/1676 [==============================] - 7s 4ms/step - loss: 0.5363 - accuracy: 0.7673
Epoch 6/30
1676/1676 [==============================] - 6s 4ms/step - loss: 0.5584 - accuracy: 0.7530
Epoch 7/30
1676/1676 [==============================] - 6s 3ms/step - loss: 0.5333 - accuracy: 0.7667
Epoch 8/30
1676/1676 [==============================] - 8s 5ms/step - loss: 0.5367 - accuracy: 0.7637
Epoch 9/30
1676/1676 [==============================] - 6s 4ms/step - loss: 0.5491 - accuracy: 0.7536
Epoch 10/30
1676/1676 [==============================] - 8s 5ms/step - loss: 0.543

In [44]:
model2.save('Trained models/LSTM #2')

In [45]:
results2 = model2.evaluate(X_test, y_test, batch_size=64)
print('test loss, test acc:', results2)

276/276 [==============================] - 0s 694us/step
test loss, test acc: [0.3917514815710593, 0.9166666865348816]


In [46]:
from sklearn.metrics import classification_report

y_pred = model2.predict(X_test, batch_size=64, verbose=1)
y_pred[y_pred <= 0.5] = 0
y_pred[y_pred > 0.5] = 1

print(classification_report(y_test, y_pred))

276/276 [==============================] - 0s 661us/step
              precision    recall  f1-score   support

         0.0       0.94      0.97      0.96       260
         1.0       0.11      0.06      0.08        16

    accuracy                           0.92       276
   macro avg       0.53      0.52      0.52       276
weighted avg       0.90      0.92      0.91       276



### 3 layers

In [15]:
n_timesteps = 10
n_features = 640

model3 = Sequential()
model3.add(LSTM(100, input_shape=(n_timesteps, n_features), return_sequences=True))
model3.add(Dropout(0.5))
model3.add(LSTM(200, return_sequences=True))
model3.add(Dropout(0.5))
model3.add(LSTM(200, return_sequences=False))
model3.add(Dropout(0.5))
model3.add(Dense(100, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))
model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 100)           296400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 100)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 200)           240800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 200)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)              

In [16]:
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

hist2 = model3.fit(X_train, y_train, epochs=30, batch_size=64, verbose=1)

Epoch 1/30
1676/1676 [==============================] - 11s 7ms/step - loss: 0.5611 - accuracy: 0.7411
Epoch 2/30
1676/1676 [==============================] - 6s 4ms/step - loss: 0.5423 - accuracy: 0.7506
Epoch 3/30
1676/1676 [==============================] - 6s 4ms/step - loss: 0.5564 - accuracy: 0.7494
Epoch 4/30
1676/1676 [==============================] - 8s 5ms/step - loss: 0.5832 - accuracy: 0.7190
Epoch 5/30
1676/1676 [==============================] - 7s 4ms/step - loss: 0.5934 - accuracy: 0.6975
Epoch 6/30
1676/1676 [==============================] - 7s 4ms/step - loss: 0.5700 - accuracy: 0.7202
Epoch 7/30
1676/1676 [==============================] - 7s 4ms/step - loss: 0.5610 - accuracy: 0.7309
Epoch 8/30
1676/1676 [==============================] - 9s 5ms/step - loss: 0.5655 - accuracy: 0.7321
Epoch 9/30
1676/1676 [==============================] - 7s 4ms/step - loss: 0.5588 - accuracy: 0.7405
Epoch 10/30
1676/1676 [==============================] - 8s 5ms/step - loss: 0.53

In [17]:
model3.save('Trained models/LSTM #3')

In [18]:
results3 = model3.evaluate(X_test, y_test, batch_size=64)
print('test loss, test acc:', results3)

276/276 [==============================] - 1s 3ms/step
test loss, test acc: [0.489266413277474, 0.9021739363670349]


In [25]:
from sklearn.metrics import classification_report

y_pred = model3.predict(X_test, batch_size=64, verbose=1)
y_pred[y_pred <= 0.5] = 0
y_pred[y_pred > 0.5] = 1

print(classification_report(y_test, y_pred))

276/276 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

         0.0       0.94      0.95      0.95       260
         1.0       0.08      0.06      0.07        16

    accuracy                           0.90       276
   macro avg       0.51      0.51      0.51       276
weighted avg       0.89      0.90      0.90       276



### Mega

In [20]:
n_timesteps = 10
n_features = 640

mega = Sequential()
mega.add(LSTM(100, input_shape=(n_timesteps, n_features), return_sequences=True))
mega.add(Dropout(0.5))
mega.add(LSTM(200, return_sequences=True))
mega.add(Dropout(0.5))
mega.add(LSTM(300, return_sequences=True))
mega.add(Dropout(0.5))
mega.add(LSTM(100, return_sequences=False))
mega.add(Dropout(0.5))
mega.add(Dense(100, activation='relu'))
mega.add(Dense(1, activation='sigmoid'))
mega.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 10, 100)           296400    
_________________________________________________________________
dropout_8 (Dropout)          (None, 10, 100)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 10, 200)           240800    
_________________________________________________________________
dropout_9 (Dropout)          (None, 10, 200)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 10, 300)           601200    
_________________________________________________________________
dropout_10 (Dropout)         (None, 10, 300)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)              

In [21]:
mega.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

hist_mega = mega.fit(X_train, y_train, epochs=30, batch_size=64, verbose=1)

Epoch 1/30
1676/1676 [==============================] - 11s 6ms/step - loss: 0.5878 - accuracy: 0.7136
Epoch 2/30
1676/1676 [==============================] - 8s 5ms/step - loss: 0.5663 - accuracy: 0.7506
Epoch 3/30
1676/1676 [==============================] - 9s 5ms/step - loss: 0.5507 - accuracy: 0.7506
Epoch 4/30
1676/1676 [==============================] - 11s 7ms/step - loss: 0.5686 - accuracy: 0.7375
Epoch 5/30
1676/1676 [==============================] - 8s 5ms/step - loss: 0.5335 - accuracy: 0.7727
Epoch 6/30
1676/1676 [==============================] - 10s 6ms/step - loss: 0.5451 - accuracy: 0.7625
Epoch 7/30
1676/1676 [==============================] - 9s 5ms/step - loss: 0.5563 - accuracy: 0.7625
Epoch 8/30
1676/1676 [==============================] - 8s 5ms/step - loss: 0.5464 - accuracy: 0.7584
Epoch 9/30
1676/1676 [==============================] - 8s 5ms/step - loss: 0.5485 - accuracy: 0.7494
Epoch 10/30
1676/1676 [==============================] - 8s 4ms/step - loss: 0.

In [22]:
mega.save('Trained models/LSTM #mega')

In [24]:
from sklearn.metrics import classification_report

y_pred = mega.predict(X_test, batch_size=64, verbose=1)
y_pred[y_pred <= 0.5] = 0
y_pred[y_pred > 0.5] = 1

print(classification_report(y_test, y_pred))

276/276 [==============================] - 1s 3ms/step
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96       260
         1.0       0.00      0.00      0.00        16

    accuracy                           0.92       276
   macro avg       0.47      0.49      0.48       276
weighted avg       0.89      0.92      0.90       276



In [26]:
results_mega = mega.evaluate(X_test, y_test, batch_size=64)
print('test loss, test acc:', results_mega)

276/276 [==============================] - 1s 3ms/step
test loss, test acc: [0.42382233384726703, 0.9202898740768433]
